In [ ]:
# extract data with release filter
import os
import csv
import pandas as pd
from pydriller import RepositoryMining
from radon.raw import analyze
from radon.metrics import h_visit
from radon.metrics import h_visit_ast
from radon.complexity import sorted_results
from pydriller.git_repository import GitRepository

fields = ['CommitID','filename','token_count','change_type','loc','lloc','sloc','comments',
          'multi','blank','code_comment','h1','h2','N1','N2','vocabulary','length',
          'calculated_length','volume', 'difficulty','effort','time','bugs']
# 'lines_added','lines_removed',
repo_updated = "/Users/sanika/Downloads/Master Thesis/repo_updated/pytest"
    
gr = GitRepository("/Users/sanika/Downloads/Master Thesis/repo_updated/pytest")
# no_commits = gitrepo.total_commits() #total number of commits

count = 0
buggy_list = []

with open('final_dataset_01.csv', 'w') as csvFile:
    writer = csv.DictWriter(csvFile, fieldnames = fields)
    writer.writeheader()
    for commit in RepositoryMining(repo_updated,
                                   only_modifications_with_file_types=['.py'] ).traverse_commits():
        for modification in commit.modifications:
            filename = modification.filename
            hash_val = commit.hash
            change_type = modification.change_type
            token_count = modification.token_count
#                 added = modification.added
#                 removed = modification.removed
            if filename.endswith(".py"):
                #Get bug inducing commits in a list
                commit1 = gr.get_commit(commit.hash)
                buggy_commits = gr.get_commits_last_modified_lines(commit1) #get bug inducing commits
                for x in buggy_commits:
                    buggy_list.append(x)
                #Calculate metrics for all commits
                for r, d, f in os.walk(repo_updated):
                    for file in f:
                        if filename in file:
                            file_path = os.path.join(r, file)
                            with open(file_path) as f:
                                content = f.read()
                                file_analyze = analyze(content)
                                code_n_comment = file_analyze.loc+file_analyze.comments
                                file_ast = h_visit(content)
                                data = [{'CommitID':(hash_val), 
                                         'filename':(file_path),
                                     'token_count':(token_count),
                                     'change_type':(change_type),
#                                      'lines_added':(added),
#                                      'lines_removed':(removed),
                                     'loc':(file_analyze.loc),
                                     'lloc':(file_analyze.lloc),
                                     'sloc':(file_analyze.sloc),
                                     'comments':(file_analyze.comments),
                                     'multi':(file_analyze.multi),
                                     'blank':(file_analyze.blank),
                                     'code_comment':(code_n_comment),
                                     'h1':(file_ast.total.h1),
                                     'h2':(file_ast.total.h2),
                                     'N1':(file_ast.total.N1),
                                     'N2':(file_ast.total.N2),
                                     'vocabulary':(file_ast.total.vocabulary),
                                     'length':(file_ast.total.length),
                                     'calculated_length':(file_ast.total.calculated_length),
                                     'volume':(file_ast.total.volume),                     
                                     'difficulty':(file_ast.total.difficulty),
                                     'effort':(file_ast.total.effort),
                                     'time':(file_ast.total.time),
                                     'bugs':(file_ast.total.bugs),}]
                                writer.writerows(data)  
print("writing completed")
csvFile.close()